# Modelo de Classificação

Import de funções da pasta de `scripts`, onde as funções e tratamento de dados estão (para todos os modelos)

In [105]:
import sys
sys.path.append('../')

import  scripts.script_data_basico
import  scripts.script_data_classificacao

reload(scripts.script_data_basico)
reload(scripts.script_data_classificacao)

tratamento = scripts.script_data_basico.tratamento
tratamento_classificacao = scripts.script_data_classificacao.tratamento_classificacao

from importlib import reload

Import de bibliotecas básicas

In [106]:
import pandas as pd
from datetime import datetime
import numpy as np

Leitura de todos os DataFrames (um para cada mês)

In [107]:
df_agosto = pd.read_csv('../data/Agosto/Ana Health_Tabela Modelo Previsão Churn - Tabela até 08_23.csv', skiprows=1)
df_julho = pd.read_csv('../data/Julho/Ana Health_Tabela Modelo Previsão Churn - Tabela até 07_23.csv', skiprows=1)
df_junho = pd.read_csv('../data/Junho/Ana Health_Tabela Modelo Previsão Churn - Tabela até 06_23.csv', skiprows=1)
df_novembro = pd.read_csv('../data/Novembro/Ana Health_Tabela Modelo Previsão Churn - Tabela Geral.csv', skiprows=1)
df_outubro = pd.read_csv('../data/Outubro/Ana Health_Tabela Modelo Previsão Churn - Tabela até 10_23.csv', skiprows=1)
df_setembro = pd.read_csv('../data/Setembro/Ana Health_Tabela Modelo Previsão Churn - Tabela até 09_23.csv', skiprows=1)

## Tratamento inicial - 

Colunas que foram Dropped:

    'state'
    'city'
    'postal_code'
    'id_person_recommendation'
    'Recebe Comunicados?'
    'Interesses'
    'Pontos de Atenção'
    'id_stage'
    'id_org'
    'status.1'
    'activities_count'
    'Datas Atendimento Médico'
    'Datas Acolhimento'
    'Datas Psicoterapia'
    'Qde Prescrições'
    'Datas Prescrição'
    'Qde Respostas WHOQOL'

**status** - Apenas utiliza o último status da amostra

**start_of_service** - Apenas contemplamos a primeira data

**lost_reason** - Apenas contemplamos a última justificativa

**Colunas de WHOQOL** - `Físico`, `Psicológico`, `Social`, `Ambiental`
- Calcula a média dos testes e substitui a amostra original

**Colunas de Data** - `start_of_service`, `add_time`, `won_time`, `lost_time.1`
- Passar tudo para DateTime e calcular o tempo da data até o dia atual

**Coluna de Tempo** - Tempo é guardado em segundos: `stay_in_pipeline_stages_welcome`,`stay_in_pipeline_stages_first_meeting`, `stay_in_pipeline_stages_whoqol`
- Transformar em dias

**contract_start_date** e **contract_end_date** - Transformação em DateTime

**Tempo até Sair** - Criação da coluna com base no **contract_start_date** e **contract_end_date**

**Tem Problema em Aberto** - Criação da Coluna.

**Data Última Mensagens Outbound** e **Data Última Mensagens Inbound** - Transformar em DateTime

**birthdate** - Transformar em Idade do Cliente

**Status Próxima Mês** - Criação da Coluna com Base no DF do Próximo Mês

In [108]:
fim_junho = datetime(2023, 6, 30)
fim_julho = datetime(2023, 7, 31)
fim_agosto = datetime(2023, 8, 31)
fim_setembro = datetime(2023, 9, 30)
fim_outubro = datetime(2023, 10, 31)
fim_novembro = datetime(2023, 11, 30)

df_agosto = tratamento(df_agosto, fim_agosto, df_setembro)
df_julho = tratamento(df_julho, fim_julho, df_agosto)
df_junho = tratamento(df_junho, fim_junho, df_julho)
df_outubro = tratamento(df_outubro, fim_outubro, df_novembro)
df_setembro = tratamento(df_setembro, fim_setembro, df_outubro)

In [109]:
df_total = pd.concat([df_junho, df_julho, df_agosto, df_setembro, df_outubro])

In [110]:
df_total.columns

Index(['id_person', 'birthdate', 'id_gender', 'id_marrital_status',
       'id_health_plan', 'contract_start_date', 'contract_end_date',
       'id_continuity_pf', 'Canal de Preferência', 'notes_count',
       'done_activities_count', 'status', 'start_of_service', 'lost_time',
       'lost_reason', 'add_time', 'id_label', 'won_time', 'lost_time.1',
       'lost_reason.1', 'Qde Todos Atendimentos', 'Faltas Todos Atendimento',
       'Qde Atendimento Médico', 'Faltas Atendimento Médico',
       'Qde Atendimentos Acolhimento', 'Faltas Acolhimento',
       'Qde Psicoterapia', 'Faltas Psicoterapia', 'Físico', 'Psicológico',
       'Social', 'Ambiental', 'Problemas Abertos', 'Mensagens Inbound',
       'Mensagens Outbound', 'Ligações Inbound',
       'Data Última Ligações Inbound', 'Ligações Outbound',
       'Data Última Ligações Outbound', 'Qde Total de Faturas',
       'Qde Total de Tentativas de Cobrança', 'Método de Pagamento',
       'Valor Médio da Mensalidade',
       'Qde Total de F

## Tratamento Específico para Classificação

Colunas que Foram Dropped:

    'id_person'
    'contract_start_date'
    'contract_end_date'
    'id_continuity_pf'
    'Canal de Preferência'
    'status'
    'lost_time'
    'add_time'
    'id_label'
    'won_time'
    'lost_time.1'
    'lost_reason'
    'lost_reason.1'
    'Qde Atendimento Médico'
    'Faltas Atendimento Médico'
    'Qde Atendimentos Acolhimento'
    'Faltas Acolhimento'
    'Qde Psicoterapia'
    'Faltas Psicoterapia'
    'Data Última Ligações Outbound'
    'Data Última Ligações Inbound'
    'Qde Total de Faturas Pagas após Vencimento'
    'Qde Perfis de Pagamento Inativos'
    'Tempo até Sair'
    'Valor Médio da Mensalidade'
    'status_prox_mes'
    'Qde Total de Faturas'
    'Problemas Abertos'

Colunas onde aplicamos o GetDummies:

    'id_gender'
    'id_marrital_status'
    'id_health_plan'
    'notes_count'
    'Método de Pagamento'
    'Qde Total de Faturas Inadimpletes'
    'Quem Enviou Última Mensagem'

Droppamos as linhas duplicadas, também.

In [111]:
df_total = tratamento_classificacao(df_total)

In [112]:
df_total.columns

Index(['idade', 'done_activities_count', 'start_of_service',
       'Qde Todos Atendimentos', 'Faltas Todos Atendimento', 'Físico',
       'Psicológico', 'Social', 'Ambiental', 'Mensagens Inbound',
       'Mensagens Outbound', 'Ligações Inbound', 'Ligações Outbound',
       'Qde Total de Tentativas de Cobrança', 'Valor Total Inadimplência',
       'Tem Problema em Aberto', 'Tempo Última Mensagem Inbound',
       'Tempo Última Mensagem Outbound', 'Target', 'id_gender_63.0',
       'id_gender_64.0', 'id_gender_Outros', 'id_marrital_status_80.0',
       'id_marrital_status_82.0', 'id_marrital_status_83.0',
       'id_health_plan_412.0', 'id_health_plan_415.0', 'id_health_plan_418.0',
       'id_health_plan_435.0', 'id_health_plan_Outros', 'notes_count_0',
       'notes_count_1', 'notes_count_2', 'notes_count_3', 'notes_count_4',
       'notes_count_5', 'notes_count_6',
       'Método de Pagamento_Cartão de crédito', 'Método de Pagamento_Dinheiro',
       'Método de Pagamento_Outros', 'Qde

### Usando Decision Tree para classificar quem sai no próximo mês

In [113]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_total.drop('Target', axis=1), df_total['Target'], test_size=0.2, random_state=42)

# Create the decision tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)

# Train the model
tree_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = tree_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.8671726755218216
Confusion Matrix:
[[ 97  23]
 [ 47 360]]
Classification Report:
              precision    recall  f1-score   support

       False       0.67      0.81      0.73       120
        True       0.94      0.88      0.91       407

    accuracy                           0.87       527
   macro avg       0.81      0.85      0.82       527
weighted avg       0.88      0.87      0.87       527



### Usando Random Forest para classificar quem sai no próximo mês

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(df_total.drop('Target', axis=1), df_total['Target'], test_size=0.2, random_state=42)

# Create the decision tree classifier
random_forest = RandomForestClassifier(random_state=42)

# Train the model
random_forest.fit(X_train, y_train)

# Make predictions on the test set
y_pred = random_forest.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(classification_rep)


Accuracy: 0.9089184060721063
Confusion Matrix:
[[ 93  27]
 [ 21 386]]
Classification Report:
              precision    recall  f1-score   support

       False       0.82      0.78      0.79       120
        True       0.93      0.95      0.94       407

    accuracy                           0.91       527
   macro avg       0.88      0.86      0.87       527
weighted avg       0.91      0.91      0.91       527



### Refazendo as Análises com CrossValidation para a realização de um TTest

In [115]:
from sklearn.model_selection import cross_val_score

# realizando a validação cruzada de cada algoritmo

random_forest_scores = cross_val_score(random_forest, X_train, y_train, scoring="accuracy", cv=10, n_jobs=-1)

tree_scores = cross_val_score(tree_clf, X_train, y_train, scoring="accuracy", cv=10, n_jobs=-1)

In [116]:
random_forest_scores

array([0.90047393, 0.90047393, 0.90995261, 0.91469194, 0.92890995,
       0.90952381, 0.9       , 0.9       , 0.9047619 , 0.87142857])

In [117]:
tree_scores

array([0.87203791, 0.8436019 , 0.8957346 , 0.89099526, 0.86729858,
       0.88095238, 0.86190476, 0.84761905, 0.9047619 , 0.8952381 ])

In [118]:
from scipy.stats import ttest_ind

# para afirmar que a hipótese nula está errada, tem que avaliar o pvalue e ver se ele é menor que o valor de alpha escolhido
# alpha escolhido foi 0.05

ttest_ind(random_forest_scores,tree_scores,equal_var=False)

Ttest_indResult(statistic=3.4589702884383247, pvalue=0.0032215963292787854)

Com esse resultado, tratamos os dois modelos como tendo resultados matematicamente diferentes. Isso significa que podemos rejeitar a hipótese de que os modelos são iguais, e prosseguir com o modelo de Random Forest como o model de clasificação que será utilizado na API.

In [119]:
import joblib

# Seu modelo treinado
modelo_treinado = random_forest

joblib.dump(modelo_treinado, '../modelos/classification_model.joblib')

['../modelos/classification_model.joblib']